In [26]:
import pandas as pd
import numpy as np
import warnings 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss

In [32]:
# Load data
games = pd.read_csv("data/MRegularSeasonCompactResults.csv")  
seeds = pd.read_csv("data/MNCAATourneySeeds.csv")
rankings = pd.read_csv("data/MMasseyOrdinals.csv")
teams = pd.read_csv("data/MTeams.csv")
game_details = pd.read_csv("data/MRegularSeasonDetailedResults.csv") 
seasons = pd.read_csv("data/MSeasons.csv")

In [33]:
# Define seasons for training and prediction
TRAIN_SEASONS = [2022, 2023, 2024]  
PREDICT_SEASON = 2025

# Get the current ranking day from 2025 season - this will be our reference point
CURRENT_RANKING_DAY = rankings[rankings['Season'] == PREDICT_SEASON]['RankingDayNum'].max()

print(f"Using RankingDayNum {CURRENT_RANKING_DAY} from 2025 season as reference point")

# Get training rankings (2022, 2023, 2024)
train_rankings = rankings[
    (rankings['Season'].isin(TRAIN_SEASONS)) & 
    (rankings['RankingDayNum'] <= CURRENT_RANKING_DAY)
]

# Get prediction rankings (2025)
predict_rankings = rankings[
    (rankings['Season'] == PREDICT_SEASON) & 
    (rankings['RankingDayNum'] <= CURRENT_RANKING_DAY)
]

# Create wide format rankings for training data - add Season to index
train_rankings_pivot = (train_rankings
    .pivot_table(
        index=['Season', 'TeamID', 'RankingDayNum'],  # Add Season to index
        columns='SystemName',
        values='OrdinalRank',
        aggfunc='first'
    )
    .reset_index()
)

# Create wide format rankings for prediction data
predict_rankings_pivot = (predict_rankings
    .pivot_table(
        index=['TeamID', 'RankingDayNum'],
        columns='SystemName',
        values='OrdinalRank',
        aggfunc='first'
    )
    .reset_index()
)

# Get seeds for each season
train_seeds = seeds[seeds["Season"].isin(TRAIN_SEASONS)]
predict_seeds = seeds[seeds["Season"] == PREDICT_SEASON]

# Print verification information
print("\nData Summary:")
print(f"Training Rankings (2022, 2023, 2024):")
print(f"- Rankings by season:")
for season in TRAIN_SEASONS:
    season_data = train_rankings[train_rankings['Season'] == season]
    print(f"\n  Season {season}:")
    print(f"  - Days available: {season_data['RankingDayNum'].nunique()}")
    print(f"  - Teams ranked: {season_data['TeamID'].nunique()}")
    print(f"  - Total rankings: {len(season_data)}")
print(f"- Ranking systems: {train_rankings['SystemName'].nunique()}")
print(f"- Total rankings across all seasons: {len(train_rankings)}")

print(f"\nPrediction Rankings (2025):")
print(f"- Days available: {predict_rankings['RankingDayNum'].nunique()}")
print(f"- Teams ranked: {predict_rankings['TeamID'].nunique()}")
print(f"- Ranking systems: {predict_rankings['SystemName'].nunique()}")
print(f"- Total rankings: {len(predict_rankings)}")

# Show sample of pivoted data
print("\nSample of Training Rankings Pivot (first 5 rows from each season):")
for season in TRAIN_SEASONS:
    print(f"\nSeason {season}:")
    print(train_rankings_pivot[train_rankings_pivot['Season'] == season].head())

print("\nColumns in Training Rankings Pivot:")
print(train_rankings_pivot.columns.tolist())

print("\nSample of Prediction Rankings Pivot (first 5 rows):")
print(predict_rankings_pivot.head())
print("\nColumns in Prediction Rankings Pivot:")
print(predict_rankings_pivot.columns.tolist())

# Optional: Check for missing values
print("\nMissing values in Training Rankings Pivot:")
print(train_rankings_pivot.isnull().sum())
print("\nMissing values in Prediction Rankings Pivot:")
print(predict_rankings_pivot.isnull().sum())

Using RankingDayNum 128 from 2025 season as reference point

Data Summary:
Training Rankings (2022, 2023, 2024):
- Rankings by season:

  Season 2022:
  - Days available: 17
  - Teams ranked: 358
  - Total rankings: 261860

  Season 2023:
  - Days available: 17
  - Teams ranked: 363
  - Total rankings: 298034

  Season 2024:
  - Days available: 18
  - Teams ranked: 362
  - Total rankings: 298730
- Ranking systems: 76
- Total rankings across all seasons: 858624

Prediction Rankings (2025):
- Days available: 18
- Teams ranked: 364
- Ranking systems: 58
- Total rankings: 281534

Sample of Training Rankings Pivot (first 5 rows from each season):

Season 2022:
SystemName  Season  TeamID  RankingDayNum    7OT  AP    ARG  ATP  BAR   BBT  \
0             2022    1101             16    NaN NaN    NaN  NaN  NaN   NaN   
1             2022    1101             23    NaN NaN    NaN  NaN  NaN  90.0   
2             2022    1101             30    NaN NaN    NaN  NaN  NaN  90.0   
3             2022  

In [34]:
# Create region mapping dictionary for each season
region_maps = {}
for season in TRAIN_SEASONS + [PREDICT_SEASON]:
    region_maps[season] = {
        'W': seasons[seasons['Season'] == season]['RegionW'].iloc[0],
        'X': seasons[seasons['Season'] == season]['RegionX'].iloc[0],
        'Y': seasons[seasons['Season'] == season]['RegionY'].iloc[0],
        'Z': seasons[seasons['Season'] == season]['RegionZ'].iloc[0]
    }

# Process seeds with region information for training seasons
train_seeds_processed = pd.DataFrame()
for season in TRAIN_SEASONS:
    season_seeds = seeds[seeds['Season'] == season].copy()
    season_seeds['SeedValue'] = season_seeds['Seed'].str.extract('(\d+)').astype(float)
    season_seeds['Region'] = season_seeds['Seed'].str[0]
    season_seeds['RegionName'] = season_seeds['Region'].map(region_maps[season])
    train_seeds_processed = pd.concat([train_seeds_processed, season_seeds])

# Process seeds with region information for prediction season
predict_seeds_processed = seeds[seeds['Season'] == PREDICT_SEASON].copy()
predict_seeds_processed['SeedValue'] = predict_seeds_processed['Seed'].str.extract('(\d+)').astype(float)
predict_seeds_processed['Region'] = predict_seeds_processed['Seed'].str[0]
predict_seeds_processed['RegionName'] = predict_seeds_processed['Region'].map(region_maps[PREDICT_SEASON])

# Add seed and region information to rankings_pivot
train_rankings_pivot = train_rankings_pivot.merge(
    train_seeds_processed[['Season', 'TeamID', 'SeedValue', 'RegionName']],
    on=['Season', 'TeamID'],
    how='left'
)

predict_rankings_pivot = predict_rankings_pivot.merge(
    predict_seeds_processed[['TeamID', 'SeedValue', 'RegionName']],
    on='TeamID',
    how='left'
)

# Print verification
print("\nRankings Data Summary:")
print(f"Training rankings shape: {train_rankings_pivot.shape}")
print(f"Prediction rankings shape: {predict_rankings_pivot.shape}")

print("\nTraining data by season:")
for season in TRAIN_SEASONS:
    season_data = train_rankings_pivot[train_rankings_pivot['Season'] == season]
    print(f"\nSeason {season}:")
    print(f"- Shape: {season_data.shape}")
    print(f"- Teams with seeds: {season_data['SeedValue'].notna().sum()}")
    print(f"- Unique regions: {season_data['RegionName'].unique().tolist()}")

print("\nSample of training rankings (first 2 rows from each season):")
for season in TRAIN_SEASONS:
    print(f"\nSeason {season}:")
    print(train_rankings_pivot[train_rankings_pivot['Season'] == season].head(2))

print("\nSample of prediction rankings (first 3 rows):")
print(predict_rankings_pivot.head(3))

print("\nColumns in training rankings:")
print(train_rankings_pivot.columns.tolist())

# Verify no missing values in key columns
print("\nMissing values in key columns (Training):")
key_cols = ['Season', 'TeamID', 'RankingDayNum', 'SeedValue', 'RegionName']
print(train_rankings_pivot[key_cols].isnull().sum())

print("\nMissing values in key columns (Prediction):")
key_cols = ['TeamID', 'RankingDayNum', 'SeedValue', 'RegionName']
print(predict_rankings_pivot[key_cols].isnull().sum())


Rankings Data Summary:
Training rankings shape: (18773, 81)
Prediction rankings shape: (6552, 62)

Training data by season:

Season 2022:
- Shape: (6086, 81)
- Teams with seeds: 1156
- Unique regions: [nan, 'East', 'West', 'South', 'Midwest']

Season 2023:
- Shape: (6171, 81)
- Teams with seeds: 1156
- Unique regions: [nan, 'South', 'West', 'Midwest', 'East']

Season 2024:
- Shape: (6516, 81)
- Teams with seeds: 1224
- Unique regions: [nan, 'Midwest', 'West', 'East', 'South']

Sample of training rankings (first 2 rows from each season):

Season 2022:
   Season  TeamID  RankingDayNum  7OT  AP  ARG  ATP  BAR   BBT    BIH  ...  \
0    2022    1101             16  NaN NaN  NaN  NaN  NaN   NaN    NaN  ...   
1    2022    1101             23  NaN NaN  NaN  NaN  NaN  90.0  190.0  ...   

     TRK    TRP  USA    WIL  WLK  WMV  WOB  WOL  SeedValue  RegionName  
0  154.0  155.0  NaN  143.0  NaN  NaN  NaN  NaN        NaN         NaN  
1  156.0  140.0  NaN  159.0  NaN  NaN  NaN  NaN        NaN   

In [35]:
print(predict_rankings_pivot.head(25))

    TeamID  RankingDayNum    7OT  AP    ARG    BAR    BBT    BIH    BMN  \
0     1101              9    NaN NaN    NaN    NaN  268.0    NaN    NaN   
1     1101             16    NaN NaN    NaN    NaN  238.0    NaN    NaN   
2     1101             23    NaN NaN    NaN    NaN  233.0  215.0    NaN   
3     1101             30  205.0 NaN  199.0    NaN  198.0  181.0    NaN   
4     1101             37  185.0 NaN  146.0    NaN  156.0  142.0    NaN   
5     1101             44  246.0 NaN  185.0    NaN  175.0  156.0  146.0   
6     1101             51  252.0 NaN    NaN    NaN    NaN  192.0  203.0   
7     1101             58  264.0 NaN    NaN    NaN  175.0  205.0  211.0   
8     1101             65  283.0 NaN  260.0    NaN  205.0  249.0  238.0   
9     1101             72  279.0 NaN  247.0    NaN  235.0  241.0  236.0   
10    1101             79  256.0 NaN    NaN  275.0  223.0  258.0  264.0   
11    1101             86  239.0 NaN    NaN    NaN  245.0  258.0  267.0   
12    1101             93

In [36]:
# Define the preferred ranking systems
preferred_systems = ["SAG", "TPR", "MAS", "RDT", "ESR", "POM", "USA", "AP"]

# Function to calculate slope with improved error handling
def calculate_slope(group):
    if len(group) < 2:  # If only one data point, return 0 slope
        return 0
    
    x = group.index.astype(float)  # RankingDayNum as x
    y = group.values
    
    # Remove any NaN or infinite values
    mask = np.isfinite(x) & np.isfinite(y)
    x = x[mask]
    y = y[mask]
    
    if len(x) < 2:  # Recheck after cleaning
        return 0
        
    # Normalize x to prevent numerical issues
    x = (x - x.min()) / (x.max() - x.min()) if x.max() != x.min() else x
    
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        coefficients = np.polyfit(x, y, 1)
    
    return coefficients[0]  # Return slope

# Function to get last n games' win rate for a specific season
def get_last_n_games_win_rate(team_id, season, n=10):
    season_games = games[games["Season"] == season]
    recent_games = season_games[
        ((season_games["WTeamID"] == team_id) | (season_games["LTeamID"] == team_id)) & 
        (season_games["DayNum"] <= CURRENT_RANKING_DAY)
    ]
    recent_games = recent_games.tail(n)
    wins = (recent_games["WTeamID"] == team_id).sum()
    return wins / n if len(recent_games) > 0 else 0.5  # Default to 50% if no games

# Function to calculate away and neutral win rates
def calculate_location_win_rates(team_id, season):
    team_games = games[
        (games['Season'] == season) &
        (games['DayNum'] <= CURRENT_RANKING_DAY) &
        ((games['WTeamID'] == team_id) | (games['LTeamID'] == team_id))
    ]
    
    away_wins = away_games = neutral_wins = neutral_games = 0
    
    for _, game in team_games.iterrows():
        if game['WLoc'] == 'A':  # Away game
            away_games += 1
            if game['WTeamID'] == team_id:
                away_wins += 1
        elif game['WLoc'] == 'N':  # Neutral game
            neutral_games += 1
            if game['WTeamID'] == team_id:
                neutral_wins += 1
    
    return pd.Series({
        'AwayWinRate': away_wins / away_games if away_games > 0 else 0.5,
        'NeutralWinRate': neutral_wins / neutral_games if neutral_games > 0 else 0.5
    })

# Initialize empty DataFrame for training slopes
train_slopes_list = []

# Process each training season separately
for season in TRAIN_SEASONS:
    # Filter rankings for this season
    season_rankings = train_rankings[train_rankings['Season'] == season]
    
    # Calculate slopes for this season
    season_slopes = (season_rankings
        .sort_values('RankingDayNum')
        .groupby(['TeamID', 'SystemName'])
        .apply(lambda x: calculate_slope(x['OrdinalRank']))
        .unstack()
        .reset_index()
    )
    
    # Add season column
    season_slopes['Season'] = season
    
    # Filter for preferred systems
    available_systems = [sys for sys in preferred_systems if sys in season_slopes.columns]
    season_slopes = season_slopes[['TeamID', 'Season'] + available_systems]
    
    # Normalize slopes for this season
    for column in available_systems:
        max_abs = np.abs(season_slopes[column]).max()
        if max_abs != 0:
            season_slopes[column] = season_slopes[column] / max_abs
    
    # Add Last10WinRate
    season_slopes['Last10WinRate'] = season_slopes['TeamID'].apply(
        lambda team: get_last_n_games_win_rate(team, season)
    )
    
    # Add Away and Neutral win rates
    location_rates = pd.DataFrame([
        calculate_location_win_rates(team_id, season)
        for team_id in season_slopes['TeamID']
    ])
    location_rates['TeamID'] = season_slopes['TeamID']
    season_slopes = season_slopes.merge(location_rates, on='TeamID', how='left')
    
    train_slopes_list.append(season_slopes)

# Combine all training seasons
train_rankings_pivot = pd.concat(train_slopes_list, ignore_index=True)

# Process prediction rankings (2025)
predict_rankings_pivot = (predict_rankings
    .sort_values('RankingDayNum')
    .groupby(['TeamID', 'SystemName'])
    .apply(lambda x: calculate_slope(x['OrdinalRank']))
    .unstack()
    .reset_index()
)

# Filter for preferred systems in prediction data
available_systems_predict = [sys for sys in preferred_systems if sys in predict_rankings_pivot.columns]
predict_rankings_pivot = predict_rankings_pivot[['TeamID'] + available_systems_predict]

# Normalize prediction slopes
for column in available_systems_predict:
    max_abs = np.abs(predict_rankings_pivot[column]).max()
    if max_abs != 0:
        predict_rankings_pivot[column] = predict_rankings_pivot[column] / max_abs

# Add win rates to prediction data
predict_rankings_pivot['Last10WinRate'] = predict_rankings_pivot['TeamID'].apply(
    lambda team: get_last_n_games_win_rate(team, PREDICT_SEASON)
)

predict_location_rates = pd.DataFrame([
    calculate_location_win_rates(team_id, PREDICT_SEASON)
    for team_id in predict_rankings_pivot['TeamID']
])
predict_location_rates['TeamID'] = predict_rankings_pivot['TeamID']
predict_rankings_pivot = predict_rankings_pivot.merge(predict_location_rates, on='TeamID', how='left')

# Add seed and region information
train_rankings_pivot = train_rankings_pivot.merge(
    train_seeds_processed[['Season', 'TeamID', 'SeedValue', 'RegionName']],
    on=['Season', 'TeamID'],
    how='left'
)

predict_rankings_pivot = predict_rankings_pivot.merge(
    predict_seeds_processed[['TeamID', 'SeedValue', 'RegionName']],
    on='TeamID',
    how='left'
)

# Organize columns
train_base_cols = ['Season', 'TeamID', 'SeedValue', 'RegionName', 'Last10WinRate', 'AwayWinRate', 'NeutralWinRate']
available_systems_train = [col for col in train_rankings_pivot.columns 
                         if col not in train_base_cols]
train_rankings_pivot = train_rankings_pivot[train_base_cols + available_systems_train]

predict_base_cols = ['TeamID', 'SeedValue', 'RegionName', 'Last10WinRate', 'AwayWinRate', 'NeutralWinRate']
predict_rankings_pivot = predict_rankings_pivot[predict_base_cols + available_systems_predict]

# Print verification
print(f"Using games up to day {CURRENT_RANKING_DAY} for calculations\n")

print("Training Rankings by Season:")
for season in TRAIN_SEASONS:
    print(f"\nSeason {season}:")
    season_data = train_rankings_pivot[train_rankings_pivot['Season'] == season]
    print(f"Shape: {season_data.shape}")
    print("\nAverage win rates:")
    print(f"Last 10: {season_data['Last10WinRate'].mean():.3f}")
    print(f"Away: {season_data['AwayWinRate'].mean():.3f}")
    print(f"Neutral: {season_data['NeutralWinRate'].mean():.3f}")
    print("\nSample data:")
    print(season_data.head(3))

print("\n2025 Prediction Rankings:")
print(f"Shape: {predict_rankings_pivot.shape}")
print("\nAverage win rates:")
print(f"Last 10: {predict_rankings_pivot['Last10WinRate'].mean():.3f}")
print(f"Away: {predict_rankings_pivot['AwayWinRate'].mean():.3f}")
print(f"Neutral: {predict_rankings_pivot['NeutralWinRate'].mean():.3f}")
print("\nSample data:")
print(predict_rankings_pivot.head(3))

# Verify no missing values
print("\nMissing values in training features:")
print(train_rankings_pivot.isnull().sum())
print("\nMissing values in prediction features:")
print(predict_rankings_pivot.isnull().sum())

/var/folders/gc/_00hl__j42163556kvkzz4b40000gq/T/ipykernel_28378/3470835442.py:77: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: calculate_slope(x['OrdinalRank']))
/var/folders/gc/_00hl__j42163556kvkzz4b40000gq/T/ipykernel_28378/3470835442.py:77: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: calculate_slope(x['OrdinalRank']))
/var/folders/gc/_00hl__j42163556kvkzz4b40000gq/T

Using games up to day 128 for calculations

Training Rankings by Season:

Season 2022:
Shape: (358, 14)

Average win rates:
Last 10: 0.501
Away: 0.510
Neutral: 0.492

Sample data:
   Season  TeamID  SeedValue RegionName  Last10WinRate  AwayWinRate  \
0    2022    1101        NaN        NaN            0.8     0.666667   
1    2022    1102        NaN        NaN            0.1     0.250000   
2    2022    1103       13.0       East            0.7     0.727273   

   NeutralWinRate       SAG       TPR       MAS       ESR       POM  USA  AP  
0        1.000000  0.103127  0.388678 -0.062486 -0.169844 -0.057897  NaN NaN  
1        0.666667 -0.204835  0.569572  0.303878  0.455674 -0.238969  NaN NaN  
2        0.333333 -0.090693  0.110091  0.171681 -0.026432  0.036306  NaN NaN  

Season 2023:
Shape: (363, 14)

Average win rates:
Last 10: 0.497
Away: 0.509
Neutral: 0.475

Sample data:
     Season  TeamID  SeedValue RegionName  Last10WinRate  AwayWinRate  \
358    2023    1101        NaN        N

In [37]:
# Create training data from games after CURRENT_RANKING_DAY for all training seasons
train_games_after = games[
    (games['Season'].isin(TRAIN_SEASONS)) & 
    (games['DayNum'] > CURRENT_RANKING_DAY)
].copy()

# Create function to format game ID string
def create_game_id(row):
    team1_id = min(row['WTeamID'], row['LTeamID'])
    team2_id = max(row['WTeamID'], row['LTeamID'])
    return f"{row['Season']:04d}_{team1_id:04d}_{team2_id:04d}"

# Add formatted game ID
train_games_after['GameID'] = train_games_after.apply(create_game_id, axis=1)

# Create new DataFrame with lower TeamID
train_games_formatted = pd.DataFrame()
train_games_formatted['GameID'] = train_games_after['GameID']
train_games_formatted['Season'] = train_games_after['Season']
train_games_formatted['TeamID'] = train_games_after.apply(lambda x: min(x['WTeamID'], x['LTeamID']), axis=1)
train_games_formatted['OpponentID'] = train_games_after.apply(lambda x: max(x['WTeamID'], x['LTeamID']), axis=1)
train_games_formatted['Outcome'] = train_games_after.apply(
    lambda x: 1 if x['WTeamID'] == min(x['WTeamID'], x['LTeamID']) else 0, 
    axis=1
)
train_games_formatted['Points_Diff'] = train_games_after.apply(
    lambda x: x['WScore'] - x['LScore'] if x['WTeamID'] == min(x['WTeamID'], x['LTeamID']) 
    else x['LScore'] - x['WScore'],
    axis=1
)

# Merge with rankings pivot for both teams using Season and TeamID
train_data = train_games_formatted.merge(
    train_rankings_pivot,
    on=['Season', 'TeamID'],
    how='left'
)

# Define columns to keep from opponent (all except Season, TeamID, RegionName)
opponent_columns = ['Season', 'TeamID'] + [col for col in train_rankings_pivot.columns 
                                         if col not in ['Season', 'TeamID', 'RegionName']]

# Define columns to calculate differences for (everything except Last10WinRate)
diff_columns = ['SeedValue', 'AwayWinRate', 'NeutralWinRate']
# Add any ranking system columns that exist
ranking_systems = ['MAS', 'ESR', 'POM', 'USA', 'AP']
diff_columns.extend([sys for sys in ranking_systems if sys in train_rankings_pivot.columns])

# Merge opponent's data
train_data = train_data.merge(
    train_rankings_pivot[opponent_columns],
    left_on=['Season', 'OpponentID'],
    right_on=['Season', 'TeamID'],
    how='left',
    suffixes=('', '_Opponent')
).drop('TeamID_Opponent', axis=1)

# Function to calculate differences with specified handling of NaN values
def calculate_diff(row, col_name):
    team_val = row[col_name]
    opp_val = row[f'{col_name}_Opponent']
    
    # If both are NaN, return NaN
    if pd.isna(team_val) and pd.isna(opp_val):
        return np.nan
    
    # Replace NaN with -16 for TeamID's value
    if pd.isna(team_val):
        team_val = -16
    
    # Replace NaN with 16 for Opponent's value
    if pd.isna(opp_val):
        opp_val = 16
    
    return team_val - opp_val

# Calculate differences for specified columns
for col in diff_columns:
    diff_col_name = f'{col}Diff'
    train_data[diff_col_name] = train_data.apply(
        lambda x: calculate_diff(x, col), axis=1
    )

# Print verification
print("Training data summary:")
print("\nGames by season:")
print(train_data['Season'].value_counts().sort_index())
print("\nTotal shape:", train_data.shape)

print("\nDifference columns created:")
diff_col_names = [f"{col}Diff" for col in diff_columns]
print(diff_col_names)

print("\nAll columns in final dataset:")
print(train_data.columns.tolist())

print("\nSample of training data from each season:")
display_cols = ['GameID', 'TeamID', 'OpponentID', 'SeedValue', 'Last10WinRate'] + diff_col_names
for season in TRAIN_SEASONS:
    print(f"\nSeason {season}:")
    print(train_data[train_data['Season'] == season][display_cols].head(2))

print("\nMissing values:")
print(train_data.isnull().sum())

# Verify all seasons are properly merged
print("\nVerification of season coverage:")
print("Unique seasons in games:", train_games_formatted['Season'].unique())
print("Unique seasons in rankings:", train_rankings_pivot['Season'].unique())
print("Unique seasons in final data:", train_data['Season'].unique())

# Print example calculations for one difference column (SeedValueDiff)
print("\nSample difference calculations for SeedValue:")
print("\nBoth teams with values:")
print(train_data[pd.notna(train_data['SeedValue']) & pd.notna(train_data['SeedValue_Opponent'])][
    ['TeamID', 'OpponentID', 'SeedValue', 'SeedValue_Opponent', 'SeedValueDiff']
].head(2))

print("\nTeamID missing value (should use -16):")
print(train_data[pd.isna(train_data['SeedValue']) & pd.notna(train_data['SeedValue_Opponent'])][
    ['TeamID', 'OpponentID', 'SeedValue', 'SeedValue_Opponent', 'SeedValueDiff']
].head(2))

Training data summary:

Games by season:
Season
2022    135
2023    121
2024    116
Name: count, dtype: int64

Total shape: (372, 37)

Difference columns created:
['SeedValueDiff', 'AwayWinRateDiff', 'NeutralWinRateDiff', 'MASDiff', 'ESRDiff', 'POMDiff', 'USADiff', 'APDiff']

All columns in final dataset:
['GameID', 'Season', 'TeamID', 'OpponentID', 'Outcome', 'Points_Diff', 'SeedValue', 'RegionName', 'Last10WinRate', 'AwayWinRate', 'NeutralWinRate', 'SAG', 'TPR', 'MAS', 'ESR', 'POM', 'USA', 'AP', 'SeedValue_Opponent', 'Last10WinRate_Opponent', 'AwayWinRate_Opponent', 'NeutralWinRate_Opponent', 'SAG_Opponent', 'TPR_Opponent', 'MAS_Opponent', 'ESR_Opponent', 'POM_Opponent', 'USA_Opponent', 'AP_Opponent', 'SeedValueDiff', 'AwayWinRateDiff', 'NeutralWinRateDiff', 'MASDiff', 'ESRDiff', 'POMDiff', 'USADiff', 'APDiff']

Sample of training data from each season:

Season 2022:
           GameID  TeamID  OpponentID  SeedValue  Last10WinRate  \
0  2022_1101_1372    1101        1372        NaN   

In [38]:
# Import required libraries
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
import numpy as np

# Define feature columns, explicitly excluding SAG and TPR
feature_cols = [col for col in train_data.columns 
                if col not in ['GameID', 'Season', 'TeamID', 'Outcome', 'RegionName', 'SAG', 'TPR', 'SAG_Opponent', 'TPR_Opponent']]

# Create X (features) and y (target)
X = train_data[feature_cols]
y = train_data['Outcome']

# First, handle missing values with SimpleImputer
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)
X_imputed = pd.DataFrame(X_imputed, columns=X.columns)

# Then scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# Create and train the logistic regression model
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_scaled, y)

# Get cross-validation scores
cv_scores = cross_val_score(lr_model, X_scaled, y, cv=5, scoring='roc_auc')

# Print model performance metrics
print("Model Performance Summary:")
print(f"\nCross-validation ROC-AUC scores:")
print(f"Mean: {cv_scores.mean():.3f} (+/- {cv_scores.std() * 2:.3f})")
print(f"Individual scores: {cv_scores}")

# Print feature importance
feature_importance = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': np.abs(lr_model.coef_[0])
})
feature_importance = feature_importance.sort_values('Importance', ascending=False)

print("\nFeatures used in model:")
print(feature_cols)

print("\nFeature Importance Rankings:")
print(feature_importance)

# Print missing value summary before imputation
print("\nMissing values before imputation:")
print(X.isnull().sum()[X.isnull().sum() > 0])

# Save important model components for later use
model_components = {
    'model': lr_model,
    'scaler': scaler,
    'imputer': imputer,
    'feature_cols': feature_cols,
    'feature_importance': feature_importance
}

Model Performance Summary:

Cross-validation ROC-AUC scores:
Mean: 0.999 (+/- 0.001)
Individual scores: [0.99928775 0.99928775 0.9992674  0.9992674  1.        ]

Features used in model:
['OpponentID', 'Points_Diff', 'SeedValue', 'Last10WinRate', 'AwayWinRate', 'NeutralWinRate', 'MAS', 'ESR', 'POM', 'USA', 'AP', 'SeedValue_Opponent', 'Last10WinRate_Opponent', 'AwayWinRate_Opponent', 'NeutralWinRate_Opponent', 'MAS_Opponent', 'ESR_Opponent', 'POM_Opponent', 'USA_Opponent', 'AP_Opponent', 'SeedValueDiff', 'AwayWinRateDiff', 'NeutralWinRateDiff', 'MASDiff', 'ESRDiff', 'POMDiff', 'USADiff', 'APDiff']

Feature Importance Rankings:
                    Feature  Importance
1               Points_Diff    5.714778
20            SeedValueDiff    0.768782
9                       USA    0.500136
17             POM_Opponent    0.296616
8                       POM    0.286231
2                 SeedValue    0.273873
18             USA_Opponent    0.270624
14  NeutralWinRate_Opponent    0.219212
21     

In [43]:
# Get all teams from predict_rankings_pivot
all_teams = predict_rankings_pivot['TeamID'].unique()

# Create all possible matchups between all teams
predict_games = []
for team1 in all_teams:
    for team2 in all_teams:
        if team1 < team2:  # Ensure each matchup is only added once
            matchup = {
                'ID': f"{PREDICT_SEASON}_{team1:04d}_{team2:04d}",
                'TeamID': team1,
                'OpponentID': team2,
                'Points_Diff': 0  # Add Points_Diff with 0 to match feature_cols
            }
            predict_games.append(matchup)

# Create prediction DataFrame
predict_games_formatted = pd.DataFrame(predict_games)

# Define columns to keep from opponent (all except TeamID, RegionName)
opponent_columns = ['TeamID'] + [col for col in predict_rankings_pivot.columns 
                               if col not in ['TeamID', 'RegionName']]

# Merge with rankings pivot for both teams using TeamID
predict_data = predict_games_formatted.merge(
    predict_rankings_pivot,
    on=['TeamID'],
    how='left'
)

# Merge opponent's data
predict_data = predict_data.merge(
    predict_rankings_pivot[opponent_columns],
    left_on=['OpponentID'],
    right_on=['TeamID'],
    how='left',
    suffixes=('', '_Opponent')
).drop('TeamID_Opponent', axis=1)

# Calculate differences for the same columns as in training
for col in diff_columns:
    diff_col_name = f'{col}Diff'
    predict_data[diff_col_name] = predict_data.apply(
        lambda x: calculate_diff(x, col), axis=1
    )

# Ensure columns match training data exactly
X_pred = predict_data[feature_cols].copy()

# Apply the same preprocessing steps as in training
X_pred_imputed = imputer.transform(X_pred)
X_pred_scaled = scaler.transform(X_pred_imputed)

# Get predictions
predict_data['Pred'] = lr_model.predict_proba(X_pred_scaled)[:, 1]

# Print results
print(f"Total teams: {len(all_teams)}")
print(f"Total possible matchups: {len(predict_data)}")

# Print available columns to debug
print("\nColumns in predict_rankings_pivot:")
print(predict_rankings_pivot.columns.tolist())

print("\nSample of predictions:")
display_cols = ['ID', 'Pred']
print(predict_data[display_cols].head(10))

print("\nPrediction distribution:")
print(predict_data['Pred'].describe())

# Save submission file
submission = predict_data[['ID', 'Pred']]
submission.to_csv('MensSubmissionStage1.csv', index=False)
print("\nSubmission saved to 'MensSubmissionStage1.csv'")

Total teams: 364
Total possible matchups: 66066

Columns in predict_rankings_pivot:
['TeamID', 'SeedValue', 'RegionName', 'Last10WinRate', 'AwayWinRate', 'NeutralWinRate', 'MAS', 'ESR', 'POM', 'USA', 'AP']

Sample of predictions:
               ID      Pred
0  2025_1101_1102  0.698634
1  2025_1101_1103  0.139395
2  2025_1101_1104  0.187837
3  2025_1101_1105  0.779813
4  2025_1101_1106  0.054197
5  2025_1101_1107  0.383863
6  2025_1101_1108  0.619557
7  2025_1101_1110  0.047648
8  2025_1101_1111  0.543609
9  2025_1101_1112  0.467434

Prediction distribution:
count    66066.000000
mean         0.473190
std          0.241656
min          0.006650
25%          0.279460
50%          0.470863
75%          0.662493
max          0.987138
Name: Pred, dtype: float64

Submission saved to 'MensSubmissionStage1.csv'


/Users/erinyepis/opt/anaconda3/envs/dbt-local-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/erinyepis/opt/anaconda3/envs/dbt-local-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [ ]:
#Run this for second submission to add upset factor for second submission

def analyze_tournament_upsets(games_df, seeds_df, rankings_df, feature_cols, imputer, scaler, lr_model, years=range(2021, 2025)):
    upset_stats = {}
    
    for seed in range(1, 11):
        total_games = 0
        upset_games = 0
        
        for year in years:
            # Get teams with this seed
            seed_teams = seeds_df[
                (seeds_df['Season'] == year) & 
                (seeds_df['SeedValue'] == seed)
            ]['TeamID'].unique()
            
            for team in seed_teams:
                # Get team's last game in the season
                last_game = games_df[
                    (games_df['Season'] == year) &
                    ((games_df['WTeamID'] == team) | (games_df['LTeamID'] == team))
                ].sort_values('DayNum').tail(1)
                
                if len(last_game) == 0:
                    continue
                
                # Format the game for prediction
                game = last_game.iloc[0]
                is_team1 = team == min(game['WTeamID'], game['LTeamID'])
                
                game_formatted = pd.DataFrame([{
                    'TeamID': min(game['WTeamID'], game['LTeamID']),
                    'OpponentID': max(game['WTeamID'], game['LTeamID']),
                    'Season': year,
                    'Points_Diff': 0  # Add Points_Diff column
                }])
                
                # Merge with rankings for both teams
                game_data = game_formatted.merge(
                    rankings_df,
                    on=['Season', 'TeamID'],
                    how='left'
                )
                
                # Define opponent columns
                opponent_columns = ['Season', 'TeamID'] + [col for col in rankings_df.columns 
                                                         if col not in ['Season', 'TeamID', 'RegionName']]
                
                # Merge opponent's data
                game_data = game_data.merge(
                    rankings_df[opponent_columns],
                    left_on=['Season', 'OpponentID'],
                    right_on=['Season', 'TeamID'],
                    how='left',
                    suffixes=('', '_Opponent')
                ).drop('TeamID_Opponent', axis=1)
                
                # Add seed values
                team_seed_data = seeds_df[
                    (seeds_df['Season'] == year) & 
                    (seeds_df['TeamID'] == game_data['TeamID'].iloc[0])
                ]
                opp_seed_data = seeds_df[
                    (seeds_df['Season'] == year) & 
                    (seeds_df['TeamID'] == game_data['OpponentID'].iloc[0])
                ]
                
                game_data['SeedValue'] = team_seed_data['SeedValue'].iloc[0] if len(team_seed_data) > 0 else np.nan
                game_data['SeedValue_Opponent'] = opp_seed_data['SeedValue'].iloc[0] if len(opp_seed_data) > 0 else np.nan
                
                # Calculate differences for the same columns as in training
                for col in diff_columns:
                    diff_col_name = f'{col}Diff'
                    game_data[diff_col_name] = game_data.apply(
                        lambda x: calculate_diff(x, col), axis=1
                    )
                
                # Print columns for debugging
                print(f"\nAvailable columns in game_data:")
                print(game_data.columns.tolist())
                print(f"\nRequired feature_cols:")
                print(feature_cols)
                
                # Get features and prediction
                X = game_data[feature_cols]
                X_imputed = imputer.transform(X)
                X_scaled = scaler.transform(X_imputed)
                pred = lr_model.predict_proba(X_scaled)[0][1]
                
                # Check if it was predicted as a win (prob > 0.6)
                if pred > 0.6:
                    total_games += 1
                    # Check if they actually lost
                    actually_won = (is_team1 and game['WTeamID'] == team) or (not is_team1 and game['LTeamID'] == team)
                    if not actually_won:
                        upset_games += 1
        
        if total_games > 0:
            upset_stats[seed] = {
                'TotalPredictedWins': total_games,
                'ActualUpsets': upset_games,
                'UpsetRate': upset_games / total_games
            }
    
    return upset_stats

# Calculate historical upset rates
upset_stats = analyze_tournament_upsets(
    games, 
    train_seeds_processed, 
    train_rankings_pivot,
    feature_cols,
    imputer,
    scaler,
    lr_model
)

# Print the upset statistics
print("\nHistorical Upset Rates for Seeds 1-10 (2021-2024):")
for seed, stats in upset_stats.items():
    print(f"\nSeed {seed}:")
    print(f"Games predicted as wins: {stats['TotalPredictedWins']}")
    print(f"Actual upsets: {stats['ActualUpsets']}")
    print(f"Upset rate: {stats['UpsetRate']:.3f}")

# Apply upset factors to 2025 predictions
def adjust_prediction_for_upsets(row, upset_stats, seeds_df):
    team_seed = seeds_df[seeds_df['TeamID'] == row['TeamID']]['SeedValue'].iloc[0] if len(seeds_df[seeds_df['TeamID'] == row['TeamID']]) > 0 else None
    
    if team_seed is not None and team_seed <= 10 and row['Pred'] > 0.6:
        upset_rate = upset_stats.get(team_seed, {}).get('UpsetRate', 0)
        # Adjust prediction based on historical upset rate
        adjusted_pred = row['Pred'] * (1 - upset_rate)
        return adjusted_pred
    
    return row['Pred']

# Apply the adjustment to tournament teams
predict_data['Pred_Adjusted'] = predict_data.apply(
    lambda x: adjust_prediction_for_upsets(x, upset_stats, predict_seeds_processed), 
    axis=1
)

# Create final submission with adjusted predictions
submission = predict_data[['ID', 'Pred_Adjusted']].rename(columns={'Pred_Adjusted': 'Pred'})
submission.to_csv('MensSubmissionStage1.csv', index=False)

# Print some comparisons
print("\nSample of original vs adjusted predictions for tournament teams:")
tournament_predictions = predict_data[
    predict_data['TeamID'].isin(predict_seeds_processed['TeamID'])
].head(10)
print(tournament_predictions[['ID', 'Pred', 'Pred_Adjusted']])


Available columns in game_data:
['TeamID', 'OpponentID', 'Season', 'Points_Diff', 'SeedValue', 'RegionName', 'Last10WinRate', 'AwayWinRate', 'NeutralWinRate', 'SAG', 'TPR', 'MAS', 'ESR', 'POM', 'USA', 'AP', 'SeedValue_Opponent', 'Last10WinRate_Opponent', 'AwayWinRate_Opponent', 'NeutralWinRate_Opponent', 'SAG_Opponent', 'TPR_Opponent', 'MAS_Opponent', 'ESR_Opponent', 'POM_Opponent', 'USA_Opponent', 'AP_Opponent', 'SeedValueDiff', 'AwayWinRateDiff', 'NeutralWinRateDiff', 'MASDiff', 'ESRDiff', 'POMDiff', 'USADiff', 'APDiff']

Required feature_cols:
['OpponentID', 'Points_Diff', 'SeedValue', 'Last10WinRate', 'AwayWinRate', 'NeutralWinRate', 'MAS', 'ESR', 'POM', 'USA', 'AP', 'SeedValue_Opponent', 'Last10WinRate_Opponent', 'AwayWinRate_Opponent', 'NeutralWinRate_Opponent', 'MAS_Opponent', 'ESR_Opponent', 'POM_Opponent', 'USA_Opponent', 'AP_Opponent', 'SeedValueDiff', 'AwayWinRateDiff', 'NeutralWinRateDiff', 'MASDiff', 'ESRDiff', 'POMDiff', 'USADiff', 'APDiff']

Available columns in game_da

/Users/erinyepis/opt/anaconda3/envs/dbt-local-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/erinyepis/opt/anaconda3/envs/dbt-local-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/erinyepis/opt/anaconda3/envs/dbt-local-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/erinyepis/opt/anaconda3/envs/dbt-local-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/erinyepis/opt/anaconda3/envs/dbt-local-env/lib/python3.11/site-packag


Available columns in game_data:
['TeamID', 'OpponentID', 'Season', 'Points_Diff', 'SeedValue', 'RegionName', 'Last10WinRate', 'AwayWinRate', 'NeutralWinRate', 'SAG', 'TPR', 'MAS', 'ESR', 'POM', 'USA', 'AP', 'SeedValue_Opponent', 'Last10WinRate_Opponent', 'AwayWinRate_Opponent', 'NeutralWinRate_Opponent', 'SAG_Opponent', 'TPR_Opponent', 'MAS_Opponent', 'ESR_Opponent', 'POM_Opponent', 'USA_Opponent', 'AP_Opponent', 'SeedValueDiff', 'AwayWinRateDiff', 'NeutralWinRateDiff', 'MASDiff', 'ESRDiff', 'POMDiff', 'USADiff', 'APDiff']

Required feature_cols:
['OpponentID', 'Points_Diff', 'SeedValue', 'Last10WinRate', 'AwayWinRate', 'NeutralWinRate', 'MAS', 'ESR', 'POM', 'USA', 'AP', 'SeedValue_Opponent', 'Last10WinRate_Opponent', 'AwayWinRate_Opponent', 'NeutralWinRate_Opponent', 'MAS_Opponent', 'ESR_Opponent', 'POM_Opponent', 'USA_Opponent', 'AP_Opponent', 'SeedValueDiff', 'AwayWinRateDiff', 'NeutralWinRateDiff', 'MASDiff', 'ESRDiff', 'POMDiff', 'USADiff', 'APDiff']

Available columns in game_da

/Users/erinyepis/opt/anaconda3/envs/dbt-local-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/erinyepis/opt/anaconda3/envs/dbt-local-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/erinyepis/opt/anaconda3/envs/dbt-local-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/erinyepis/opt/anaconda3/envs/dbt-local-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/erinyepis/opt/anaconda3/envs/dbt-local-env/lib/python3.11/site-packag


Available columns in game_data:
['TeamID', 'OpponentID', 'Season', 'Points_Diff', 'SeedValue', 'RegionName', 'Last10WinRate', 'AwayWinRate', 'NeutralWinRate', 'SAG', 'TPR', 'MAS', 'ESR', 'POM', 'USA', 'AP', 'SeedValue_Opponent', 'Last10WinRate_Opponent', 'AwayWinRate_Opponent', 'NeutralWinRate_Opponent', 'SAG_Opponent', 'TPR_Opponent', 'MAS_Opponent', 'ESR_Opponent', 'POM_Opponent', 'USA_Opponent', 'AP_Opponent', 'SeedValueDiff', 'AwayWinRateDiff', 'NeutralWinRateDiff', 'MASDiff', 'ESRDiff', 'POMDiff', 'USADiff', 'APDiff']

Required feature_cols:
['OpponentID', 'Points_Diff', 'SeedValue', 'Last10WinRate', 'AwayWinRate', 'NeutralWinRate', 'MAS', 'ESR', 'POM', 'USA', 'AP', 'SeedValue_Opponent', 'Last10WinRate_Opponent', 'AwayWinRate_Opponent', 'NeutralWinRate_Opponent', 'MAS_Opponent', 'ESR_Opponent', 'POM_Opponent', 'USA_Opponent', 'AP_Opponent', 'SeedValueDiff', 'AwayWinRateDiff', 'NeutralWinRateDiff', 'MASDiff', 'ESRDiff', 'POMDiff', 'USADiff', 'APDiff']

Available columns in game_da

/Users/erinyepis/opt/anaconda3/envs/dbt-local-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/erinyepis/opt/anaconda3/envs/dbt-local-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/erinyepis/opt/anaconda3/envs/dbt-local-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/erinyepis/opt/anaconda3/envs/dbt-local-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/erinyepis/opt/anaconda3/envs/dbt-local-env/lib/python3.11/site-packag


Sample of original vs adjusted predictions for tournament teams:
                 ID      Pred  Pred_Adjusted
725  2025_1103_1104  0.927970       0.927970
726  2025_1103_1105  0.966204       0.966204
727  2025_1103_1106  0.761448       0.761448
728  2025_1103_1107  0.834141       0.834141
729  2025_1103_1108  0.929309       0.929309
730  2025_1103_1110  0.735936       0.735936
731  2025_1103_1111  0.905794       0.905794
732  2025_1103_1112  0.979956       0.979956
733  2025_1103_1113  0.890645       0.890645
734  2025_1103_1114  0.859676       0.859676
